In [ ]:
import datetime
from pathlib import Path

import imageio.v2 as imageio
import numpy as np
import pandas as pd
import rasterio
import tqdm
from PIL import Image, ImageDraw, ImageFont

from estuary.model.data import parse_dt_from_pth
from estuary.util import broad_band, false_color

In [ ]:
BASE = Path("/Users/kyledorman/data/results/estuary/train/20251008-151833")
preds = pd.read_csv(BASE / "timeseries_preds.csv")
preds["acquired"] = preds.source_tif.apply(lambda a: parse_dt_from_pth(Path(a)))
preds = preds.sort_values(by="acquired")

In [ ]:
def draw_label(
    img: Image.Image, text: str, color: tuple[int, int, int], add_border=True
) -> Image.Image:
    """Draw a semi-transparent banner with outlined text, and optional colored border."""
    # Optional: try a nicer font; fall back to default if not available
    try:
        FONT = ImageFont.truetype("/System/Library/Fonts/Supplemental/Arial Bold.ttf", 20)
    except Exception:
        FONT = ImageFont.load_default()

    draw = ImageDraw.Draw(img, "RGBA")
    w, h = img.size

    # Banner box
    pad_x, pad_y = 10, 8
    text_w, text_h = draw.textbbox((0, 0), text, font=FONT)[2:]
    box_w = min(w - 2 * pad_x, text_w + 2 * pad_x)
    box_h = text_h + 2 * pad_y

    # Top-left anchor for banner
    x0, y0 = pad_x, pad_y
    x1, y1 = x0 + box_w, y0 + box_h

    # Semi-transparent dark banner
    draw.rounded_rectangle([x0, y0, x1, y1], radius=10, fill=(0, 0, 0, 110))

    # Outlined text (stroke) for readability
    draw.text(
        (x0 + pad_x, y0 + pad_y),
        text,
        font=FONT,
        fill=(255, 255, 255, 255),
        stroke_width=2,
        stroke_fill=(0, 0, 0, 220),
    )

    # Optional border matching class color
    if add_border:
        draw.rectangle([0, 0, w - 1, h - 1], outline=color + (255,), width=4)

    return img

In [ ]:
sorted(preds.region.unique().tolist())

In [ ]:
region = 48
start = datetime.datetime(year=2023, month=10, day=1)
end = datetime.datetime(year=2024, month=2, day=1)

gif_df = preds[(preds.region == region) & (preds.acquired > start) & (preds.acquired < end)].copy()

len(gif_df)

In [ ]:
save_path = Path(f"/Users/kyledorman/Desktop/{region}_{start.date()}.mp4")
save_path.parent.mkdir(exist_ok=True, parents=True)

frames = []
size = None
for _, row in tqdm.tqdm(gif_df.iterrows(), total=len(gif_df)):
    pth = row.source_tif
    pred_name = "open" if row.y_pred == 1 else "close"
    pred_color = (44, 160, 44) if row.y_pred == 1 else (214, 39, 40)  # green/red
    date_str = getattr(row, "acquired", None)
    if date_str is not None:
        # Parse YYYYMMDD or ISO-like strings robustly
        try:
            # if already datetime-like, this is a no-op; else try %Y%m%d
            dt = pd.to_datetime(date_str, format="%Y%m%d", errors="ignore")
            dt = pd.to_datetime(dt)  # ensure Timestamp
            date_disp = dt.strftime("%Y-%m-%d")
        except Exception:
            date_disp = str(date_str)
    else:
        date_disp = ""

    with rasterio.open(pth) as src:
        data = src.read(out_dtype=np.float32)
        nodata = src.read(1, masked=True).mask
        if len(data) == 8:
            img = broad_band(data, nodata)
        else:
            img = false_color(data, nodata)
    img = Image.fromarray(img)
    if size is None:
        size = img.size
        size = tuple([2 * (s // 2) for s in size])
    img = img.resize(size)

    # Compose label text — include region/pred/conf/date as you like
    label_text = f"{pred_name}"
    if date_disp:
        label_text = f"{date_disp} • " + label_text

    img = draw_label(img, label_text, pred_color, add_border=True)

    frames.append(img)

# Convert each PIL frame to a NumPy array (imageio needs ndarray or PIL)
frame_arrays = [np.array(im.convert("RGB")) for im in frames]

len(frame_arrays)

In [ ]:
# Write MP4 (H.264)
fps = 1
imageio.mimsave(
    save_path,
    frame_arrays,
    fps=fps,
    codec="libx264",  # H.264 for compatibility
    quality=10,  # 0 (lowest) - 10 (highest) for libx264
    macro_block_size=None,  # keeps original frame size
)
print(f"Saved video → {save_path}")

In [ ]:
from IPython.display import Video

Video(str(save_path), embed=True, width=600)